resnet18  
fcn

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import pickle
import numpy as np

# 데이터셋 크기에 맞는 CNN 모델 불러오기 (예: ResNet18)
model1 = models.resnet18(pretrained=True)

# 모델의 마지막 레이어를 새로운 fully connected layer로 교체
num_features = model1.fc.in_features
model1.fc = nn.Linear(num_features, 2)  # 예시에서는 클래스가 2개라 가정

# GPU 사용이 가능하다면 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = model1.to(device)

# Loss function 및 optimizer 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model1.parameters(), lr=0.001, momentum=0.9)

# 학습 데이터 및 레이블
xTrainName = "xTrain.pkl"
yTrainName = "yTrain.pkl"

with open(xTrainName,'rb') as f1:
    X = pickle.load(f1)

with open(yTrainName,'rb') as f2:
    y = pickle.load(f2)

x_np = X.to_numpy()
y_np = y.to_numpy()

# 데이터를 PyTorch 텐서로 변환
x_train = torch.tensor(np.tile(x_np, (2, 1, 1, 1)), dtype=torch.float32)  # 배치 차원 추가하여 [배치 크기, 채널 수(여기서는 1), 높이, 너비]
x_train = x_train.expand(-1, 3, -1, -1)  # 1개의 채널을 3개의 채널로 확장
y_train = torch.tensor(np.tile(y_np, (2,)), dtype=torch.long)  # 배치 차원 추가하지 않음

# 학습
num_epochs = 10
for epoch in range(num_epochs):
    model1.train()
    running_loss = 0.0
    for inputs, labels in zip(x_train, y_train):
        optimizer.zero_grad()
        inputs = inputs.unsqueeze(0).to(device)  # 배치 차원 추가
        outputs = model1(inputs)
        loss = criterion(outputs, labels.unsqueeze(0).to(device))  # 배치 차원 추가
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(x_train)}")

print("학습 완료")

c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.7219103872776031
Epoch [2/10], Loss: 0.2060864269733429
Epoch [3/10], Loss: 0.03430333733558655
Epoch [4/10], Loss: 0.006679045967757702
Epoch [5/10], Loss: 0.0017442298703826964
Epoch [6/10], Loss: 0.000585255867918022
Epoch [7/10], Loss: 0.0002408322543487884
Epoch [8/10], Loss: 0.00011699690003297292
Epoch [9/10], Loss: 6.526492143166251e-05
Epoch [10/10], Loss: 4.070913382747676e-05
학습 완료


In [3]:
correct = 0
total = 0

# 모델을 evaluation 모드로 설정
model1.eval()

# 모델이 예측한 결과와 실제 레이블을 비교하여 정확도 계산
with torch.no_grad():
    for inputs, labels in zip(x_train, y_train):
        inputs = inputs.unsqueeze(0).to(device)
        outputs = model1(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels.to(device)).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%


resnet18  
1d conv layer

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import pickle
import numpy as np

# 데이터셋 크기에 맞는 CNN 모델 불러오기 (예: ResNet18)
model2 = models.resnet18(pretrained=True)

# 모델의 마지막 레이어를 변경하고자 할 경우
# 모델의 마지막 레이어 이전의 출력 차원을 확인
num_features = model2.fc.in_features

# 새로운 모델 정의
class CustomModel(nn.Module):
    def __init__(self, num_features):
        super(CustomModel, self).__init__()
        self.features = nn.Sequential(*list(model2.children())[:-2])  # ResNet의 마지막 2개 레이어 제외
        self.conv1d = nn.Conv1d(num_features, 64, kernel_size=3, stride=1, padding=1)  # 1D Convolutional Layer 추가
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(64, 2)  # 예시에서는 클래스가 2개라 가정

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), num_features, -1)  # Flatten 작업
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)  # 1D로 평탄화
        x = self.fc(x)
        return x

# 새로운 모델 생성
model2 = CustomModel(num_features)

# GPU 사용이 가능하다면 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2 = model2.to(device)

# Loss function 및 optimizer 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)

# 학습 데이터 및 레이블
xTrainName = "xTrain.pkl"
yTrainName = "yTrain.pkl"

with open(xTrainName,'rb') as f1:
    X = pickle.load(f1)

with open(yTrainName,'rb') as f2:
    y = pickle.load(f2)

x_np = X.to_numpy()
y_np = y.to_numpy()

# 데이터를 PyTorch 텐서로 변환
x_train = torch.tensor(np.tile(x_np, (2, 1, 1, 1)), dtype=torch.float32)  # 배치 차원 추가하여 [배치 크기, 채널 수(여기서는 1), 높이, 너비]
x_train = x_train.expand(-1, 3, -1, -1)  # 1개의 채널을 3개의 채널로 확장
y_train = torch.tensor(np.tile(y_np, (2,)), dtype=torch.long)  # 배치 차원 추가하지 않음

# 학습
num_epochs = 10
for epoch in range(num_epochs):
    model2.train()
    running_loss = 0.0
    for inputs, labels in zip(x_train, y_train):
        optimizer.zero_grad()
        inputs = inputs.unsqueeze(0).to(device)  # 배치 차원 추가
        outputs = model2(inputs)
        loss = criterion(outputs, labels.unsqueeze(0).to(device))  # 배치 차원 추가
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(x_train)}")

print("학습 완료")


c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.5772333741188049
Epoch [2/10], Loss: 0.3981121927499771
Epoch [3/10], Loss: 0.21757658571004868
Epoch [4/10], Loss: 0.1055511012673378
Epoch [5/10], Loss: 0.048364829272031784
Epoch [6/10], Loss: 0.022694178856909275
Epoch [7/10], Loss: 0.011362233199179173
Epoch [8/10], Loss: 0.006183403776958585
Epoch [9/10], Loss: 0.0036628663074225187
Epoch [10/10], Loss: 0.0023467119317501783
학습 완료


In [5]:
correct = 0
total = 0

# 모델을 evaluation 모드로 설정
model2.eval()

# 모델이 예측한 결과와 실제 레이블을 비교하여 정확도 계산
with torch.no_grad():
    for inputs, labels in zip(x_train, y_train):
        inputs = inputs.unsqueeze(0).to(device)
        outputs = model2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels.to(device)).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%


resnet18  
2d conv layer

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import pickle
import numpy as np

# 데이터셋 크기에 맞는 CNN 모델 불러오기 (예: ResNet18)
model3 = models.resnet18(pretrained=True)

# 모델의 마지막 레이어를 변경하고자 할 경우
# 모델의 마지막 레이어 이전의 출력 차원을 확인
num_features = model3.fc.in_features

# 새로운 모델 정의
class CustomModel(nn.Module):
    def __init__(self, num_features):
        super(CustomModel, self).__init__()
        self.features = nn.Sequential(*list(model3.children())[:-2])  # ResNet의 마지막 2개 레이어 제외
        self.conv2d = nn.Conv2d(num_features, 64, kernel_size=3, stride=1, padding=1)  # 2D Convolutional Layer 추가
        self.relu = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, 2)  # 예시에서는 클래스가 2개라 가정

    def forward(self, x):
        x = self.features(x)
        x = self.conv2d(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)  # 2D 출력을 1D로 평탄화
        x = self.fc(x)
        return x

# 새로운 모델 생성
model3 = CustomModel(num_features)

# GPU 사용이 가능하다면 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model3 = model3.to(device)

# Loss function 및 optimizer 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model3.parameters(), lr=0.001, momentum=0.9)

# 학습 데이터 및 레이블
xTrainName = "xTrain.pkl"
yTrainName = "yTrain.pkl"

with open(xTrainName,'rb') as f1:
    X = pickle.load(f1)

with open(yTrainName,'rb') as f2:
    y = pickle.load(f2)

x_np = X.to_numpy()
y_np = y.to_numpy()

# 데이터를 PyTorch 텐서로 변환
x_train = torch.tensor(np.tile(x_np, (2, 1, 1, 1)), dtype=torch.float32)  # 배치 차원 추가하여 [배치 크기, 채널 수(여기서는 1), 높이, 너비]
x_train = x_train.expand(-1, 3, -1, -1)  # 1개의 채널을 3개의 채널로 확장
y_train = torch.tensor(np.tile(y_np, (2,)), dtype=torch.long)  # 배치 차원 추가하지 않음

# 학습
num_epochs = 10
for epoch in range(num_epochs):
    model3.train()
    running_loss = 0.0
    for inputs, labels in zip(x_train, y_train):
        optimizer.zero_grad()
        inputs = inputs.unsqueeze(0).to(device)  # 배치 차원 추가
        outputs = model3(inputs)
        loss = criterion(outputs, labels.unsqueeze(0).to(device))  # 배치 차원 추가
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(x_train)}")

print("학습 완료")


c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bluecom014\miniconda3\envs\sesac\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.5847071707248688
Epoch [2/10], Loss: 0.37857693433761597
Epoch [3/10], Loss: 0.17081908881664276
Epoch [4/10], Loss: 0.06395904161036015
Epoch [5/10], Loss: 0.023804597556591034
Epoch [6/10], Loss: 0.009719994850456715
Epoch [7/10], Loss: 0.004481855663470924
Epoch [8/10], Loss: 0.002331458032131195
Epoch [9/10], Loss: 0.0013498947373591363
Epoch [10/10], Loss: 0.0008576967229600996
학습 완료


In [7]:
correct = 0
total = 0

# 모델을 evaluation 모드로 설정
model3.eval()

# 모델이 예측한 결과와 실제 레이블을 비교하여 정확도 계산
with torch.no_grad():
    for inputs, labels in zip(x_train, y_train):
        inputs = inputs.unsqueeze(0).to(device)
        outputs = model3(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += 1
        correct += (predicted == labels.to(device)).sum().item()

accuracy = correct / total
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 100.00%
